### Time Series Analysis for RND Trace VM
###### http://gwa.ewi.tudelft.nl/datasets/gwa-t-12-bitbrains

### This Notebook contains models Auto Regressive and Long Short Term Memory

In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
from statsmodels.tsa.stattools import adfuller

#### Combining CSV Files

In [13]:
import os
import glob
import pandas as pd
os.chdir("C:/Users/jahna/Downloads/data")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_3.csv", index=False, encoding='utf-8-sig')

In [14]:
import csv

with open('C:/Users/jahna/Downloads/data/combined_3.csv', 'r') as inf, open('C:/Users/jahna/Downloads/data/data_3.csv', 'w') as of:

    r = csv.reader(inf, delimiter=';')
    w = csv.writer(of, delimiter=',')

    for line in r:
        trim = (field.strip('	') for field in line)
        w.writerow(trim)

#### Reading the processed data into dataframe called 'data'

In [15]:
data = pd.read_csv('C:/Users/jahna/Downloads/data/data_3.csv')

#### Converting Epoch to Date Time Format

In [16]:
data['Timestamp [ms]'] = pd.to_datetime(data['Timestamp [ms]'], unit='s')

In [17]:
data.head()

,Timestamp [ms],CPU cores,CPU capacity provisioned [MHZ],CPU usage [MHZ],CPU usage [%],Memory capacity provisioned [KB],Memory usage [KB],Disk read throughput [KB/s],Disk write throughput [KB/s],Network received throughput [KB/s],Network transmitted throughput [KB/s]
0,2013-06-30 22:03:24,1,2599.999398,25.999994,1.000000,702948.0,92272.800000,0.0,4.333333,0.066667,0.066667
1,2013-06-30 22:08:24,1,2599.999398,25.999994,1.000000,702948.0,71300.800000,0.0,4.266667,0.066667,0.066667
2,2013-06-30 22:13:24,1,2599.999398,24.266661,0.933333,702948.0,72698.933333,0.0,4.000000,0.133333,0.133333
3,2013-06-30 22:18:24,1,2599.999398,25.999994,1.000000,702948.0,67106.400000,0.0,4.600000,0.066667,0.133333
4,2013-06-30 22:23:24,1,2599.999398,25.999994,1.000000,702948.0,64310.133333,0.0,4.066667,0.133333,0.133333


In [18]:
data.tail()

,Timestamp [ms],CPU cores,CPU capacity provisioned [MHZ],CPU usage [MHZ],CPU usage [%],Memory capacity provisioned [KB],Memory usage [KB],Disk read throughput [KB/s],Disk write throughput [KB/s],Network received throughput [KB/s],Network transmitted throughput [KB/s]
25184,2013-09-29 21:38:50,1,2599.999304,25.999993,1.0,702948.0,97865.333333,0.0,5.400000,3.000000,2.000000
25185,2013-09-29 21:43:50,1,2599.999304,25.999993,1.0,702948.0,74097.066667,0.0,4.800000,3.000000,2.000000
25186,2013-09-29 21:48:50,1,2599.999304,25.999993,1.0,702948.0,88078.400000,0.0,5.333333,3.000000,2.000000
25187,2013-09-29 21:53:50,1,2599.999304,25.999993,1.0,702948.0,68504.533333,0.0,4.600000,3.000000,2.000000
25188,2013-09-29 21:58:50,1,2599.999304,25.999993,1.0,702948.0,58717.600000,0.0,4.866667,2.933333,1.933333


In [19]:
data=data.set_index('Timestamp [ms]')

#### Calculating Memory Usage %

In [20]:
data['Memory usage [%]']=(data['Memory usage [KB]'].div(data['Memory capacity provisioned [KB]']))*100

In [21]:
data.head()

,CPU cores,CPU capacity provisioned [MHZ],CPU usage [MHZ],CPU usage [%],Memory capacity provisioned [KB],Memory usage [KB],Disk read throughput [KB/s],Disk write throughput [KB/s],Network received throughput [KB/s],Network transmitted throughput [KB/s],Memory usage [%]
Timestamp [ms],,,,,,,,,,,
2013-06-30 22:03:24,1,2599.999398,25.999994,1.000000,702948.0,92272.800000,0.0,4.333333,0.066667,0.066667,13.126547
2013-06-30 22:08:24,1,2599.999398,25.999994,1.000000,702948.0,71300.800000,0.0,4.266667,0.066667,0.066667,10.143112
2013-06-30 22:13:24,1,2599.999398,24.266661,0.933333,702948.0,72698.933333,0.0,4.000000,0.133333,0.133333,10.342007
2013-06-30 22:18:24,1,2599.999398,25.999994,1.000000,702948.0,67106.400000,0.0,4.600000,0.066667,0.133333,9.546424
2013-06-30 22:23:24,1,2599.999398,25.999994,1.000000,702948.0,64310.133333,0.0,4.066667,0.133333,0.133333,9.148633


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25189 entries, 2013-06-30 22:03:24 to 2013-09-29 21:58:50
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   CPU cores                              25189 non-null  int64  
 1   CPU capacity provisioned [MHZ]         25189 non-null  float64
 2   CPU usage [MHZ]                        25189 non-null  float64
 3   CPU usage [%]                          25189 non-null  float64
 4   Memory capacity provisioned [KB]       25189 non-null  float64
 5   Memory usage [KB]                      25189 non-null  float64
 6   Disk read throughput [KB/s]            25189 non-null  float64
 7   Disk write throughput [KB/s]           25189 non-null  float64
 8   Network received throughput [KB/s]     25189 non-null  float64
 9   Network transmitted throughput [KB/s]  25189 non-null  float64
 10  Memory usage [%]                   

In [23]:
data.isnull().sum()

CPU cores                                0
CPU capacity provisioned [MHZ]           0
CPU usage [MHZ]                          0
CPU usage [%]                            0
Memory capacity provisioned [KB]         0
Memory usage [KB]                        0
Disk read throughput [KB/s]              0
Disk write throughput [KB/s]             0
Network received throughput [KB/s]       0
Network transmitted throughput [KB/s]    0
Memory usage [%]                         0
dtype: int64

In [24]:
data.describe()

,CPU cores,CPU capacity provisioned [MHZ],CPU usage [MHZ],CPU usage [%],Memory capacity provisioned [KB],Memory usage [KB],Disk read throughput [KB/s],Disk write throughput [KB/s],Network received throughput [KB/s],Network transmitted throughput [KB/s],Memory usage [%]
count,25189.0,25189.000000,25189.000000,25189.000000,25189.0,25189.000000,25189.000000,25189.000000,25189.000000,25189.000000,25189.000000
mean,1.0,2599.999257,25.877720,0.995297,702948.0,74347.350399,0.000003,4.331133,1.897198,1.048559,10.576508
std,0.0,0.000179,0.846818,0.032570,0.0,16444.247285,0.000420,0.451114,1.056421,0.314386,2.339326
min,1.0,2599.998967,13.866661,0.533333,702948.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,1.0,2599.999006,25.999990,1.000000,702948.0,62912.000000,0.000000,4.066667,1.000000,1.000000,8.949737
50%,1.0,2599.999309,25.999993,1.000000,702948.0,74097.066667,0.000000,4.266667,2.000000,1.000000,10.540903
75%,1.0,2599.999334,25.999993,1.000000,702948.0,83884.000000,0.000000,4.533333,2.000000,1.000000,11.933173
max,1.0,2599.999758,51.999980,2.000000,702948.0,167771.200000,0.066667,25.133333,5.200000,4.533333,23.866801


In [25]:
data.shape

(25189, 11)

##### Auto Regressive Model for CPU usage %

In [26]:
from statsmodels.tsa.ar_model import AutoReg

In [27]:
cdata = data[['CPU usage [%]']]
cdata.head()

,CPU usage [%]
Timestamp [ms],
2013-06-30 22:03:24,1.000000
2013-06-30 22:08:24,1.000000
2013-06-30 22:13:24,0.933333
2013-06-30 22:18:24,1.000000
2013-06-30 22:23:24,1.000000


In [28]:
x1 = cdata.values
len(x1)

25189

#### Splitting CPU usage % data into train and test split

In [29]:
train = x1[:len(x1)-5000]
len(train)

20189

In [30]:
test = x1[len(x1)-5000:]
len(test)

5000

#### Building AutoRegressive model on the train data and predicted using test data

In [31]:
model=AutoReg(train, lags=47).fit()
predcpu=model.predict(start=len(train), end=len(x1)-1, dynamic=False)

C:\Users\jahna\anaconda3\envs\env\lib\site-packages\statsmodels\tsa\ar_model.py:252: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning,


In [32]:
len(predcpu)

5000

#### Evaluation Metrics

##### Root Mean Square Error

In [33]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [34]:
rmse=sqrt(mean_squared_error(test,predcpu))
rmse

0.032131091355338974

##### Mean Absolute Error

In [22]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(test,predcpu)

0.010685267785590116

#### Forecasting CPU usage % for the next three months

In [35]:
forecastcpu=model.predict(start=len(x1)-1, end=51108, dynamic=False)

In [36]:
len(forecastcpu)

25921

In [37]:
predictedcpu = pd.DataFrame(forecastcpu, columns=['predicted_cpu_usage [%]'])
len(predictedcpu)

25921

#### Creating timestamp for the next three months

In [39]:
time = pd.date_range('2013-09-29 22:03:50', periods=25921, freq='5min')
time

DatetimeIndex(['2013-09-29 22:03:50', '2013-09-29 22:08:50',
               '2013-09-29 22:13:50', '2013-09-29 22:18:50',
               '2013-09-29 22:23:50', '2013-09-29 22:28:50',
               '2013-09-29 22:33:50', '2013-09-29 22:38:50',
               '2013-09-29 22:43:50', '2013-09-29 22:48:50',
               ...
               '2013-12-28 21:18:50', '2013-12-28 21:23:50',
               '2013-12-28 21:28:50', '2013-12-28 21:33:50',
               '2013-12-28 21:38:50', '2013-12-28 21:43:50',
               '2013-12-28 21:48:50', '2013-12-28 21:53:50',
               '2013-12-28 21:58:50', '2013-12-28 22:03:50'],
              dtype='datetime64[ns]', length=25921, freq='5T')

In [40]:
predictedcpu['Timestamp']= time

In [41]:
predictedcpu.head()

,predicted_cpu_usage [%],Timestamp
0,0.994962,2013-09-29 22:03:50
1,0.994962,2013-09-29 22:08:50
2,0.994962,2013-09-29 22:13:50
3,0.994962,2013-09-29 22:18:50
4,0.994962,2013-09-29 22:23:50


##### Auto Regressive Model for Memory usage %

In [42]:
mdata = data[['Memory usage [%]']]
mdata.head()

,Memory usage [%]
Timestamp [ms],
2013-06-30 22:03:24,13.126547
2013-06-30 22:08:24,10.143112
2013-06-30 22:13:24,10.342007
2013-06-30 22:18:24,9.546424
2013-06-30 22:23:24,9.148633


In [43]:
x2 = mdata.values
len(x2)

25189

#### Splitting Memory usage % data into train and test split

In [46]:
memtrain = x2[:len(x2)-5000]
memtrain

array([[13.12654706],
       [10.14311158],
       [10.34200728],
       ...,
       [11.73427717],
       [11.73427717],
       [ 9.14863309]])

In [47]:
memtest = x2[len(x2)-5000:]
memtest

array([[ 9.54642449],
       [ 8.94973739],
       [ 9.54642449],
       ...,
       [12.52985996],
       [ 9.74532018],
       [ 8.3530503 ]])

#### Building AutoRegressive model on the train data and predicted using test data

In [48]:
model=AutoReg(memtrain, lags=47).fit()
predmem=model.predict(start=len(memtrain), end=len(x1)-1, dynamic=False)

C:\Users\jahna\anaconda3\envs\env\lib\site-packages\statsmodels\tsa\ar_model.py:252: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  FutureWarning,


#### Evaluation Metrics

##### RMSE

In [34]:
rmse=sqrt(mean_squared_error(memtest,predmem))
rmse

2.3600077513934545

#### MAE

In [49]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(memtest,predmem)

1.8509603277485243

#### Forecasting Memory usage % for the next three months

In [51]:
forecastmem=model.predict(start=len(x1)-1, end=51108, dynamic=False)

In [52]:
len(forecastmem)

25921

In [53]:
predictedmem = pd.DataFrame(forecastmem, columns=['predicted_memory_usage [%]'])

In [54]:
predictedmem['Timestamp'] = time

#### Joining both predictions of memory and CPU to one dataframe

In [55]:
import pandas as pd
finalar = pd.merge(predictedcpu, predictedmem, on='Timestamp', how='outer')

In [56]:
finalar.head()

,predicted_cpu_usage [%],Timestamp,predicted_memory_usage [%]
0,0.994962,2013-09-29 22:03:50,10.515294
1,0.994962,2013-09-29 22:08:50,10.515294
2,0.994962,2013-09-29 22:13:50,10.515294
3,0.994962,2013-09-29 22:18:50,10.515294
4,0.994962,2013-09-29 22:23:50,10.515294


In [57]:
finalar['CPU capacity provisioned [MHZ]'] = 2599.999304

In [58]:
finalar['Memory capacity provisioned [KB]'] = 702948.0

In [59]:
finalar['CPU usage [MHZ]']=(finalar['predicted_cpu_usage [%]'].mul(finalar['CPU capacity provisioned [MHZ]']/100))

In [60]:
finalar['Memory usage [KB]']=(finalar['predicted_memory_usage [%]'].mul(finalar['Memory capacity provisioned [KB]']/100))

In [61]:
finalar['RMSE_CPU'] = 0.03212

In [62]:
finalar['MAE_CPU'] = 0.0107

In [63]:
finalar['RMSE_Memory'] = 2.35996

In [64]:
finalar['MAE_Memory'] = 1.85096

In [65]:
finalar['Model'] = 'AutoRegressive'

In [66]:
finalar['VM'] =3

In [67]:
finalar['CPU cores'] =2

In [68]:
finalar

,predicted_cpu_usage [%],Timestamp,predicted_memory_usage [%],CPU capacity provisioned [MHZ],Memory capacity provisioned [KB],CPU usage [MHZ],Memory usage [KB],RMSE_CPU,MAE_CPU,RMSE_Memory,MAE_Memory,Model,VM,CPU cores
0,0.994962,2013-09-29 22:03:50,10.515294,2599.999304,702948.0,25.869017,73917.050931,0.03212,0.0107,2.35996,1.85096,AutoRegressive,3,2
1,0.994962,2013-09-29 22:08:50,10.515294,2599.999304,702948.0,25.869017,73917.050931,0.03212,0.0107,2.35996,1.85096,AutoRegressive,3,2
2,0.994962,2013-09-29 22:13:50,10.515294,2599.999304,702948.0,25.869017,73917.050931,0.03212,0.0107,2.35996,1.85096,AutoRegressive,3,2
3,0.994962,2013-09-29 22:18:50,10.515294,2599.999304,702948.0,25.869017,73917.050931,0.03212,0.0107,2.35996,1.85096,AutoRegressive,3,2
4,0.994962,2013-09-29 22:23:50,10.515294,2599.999304,702948.0,25.869017,73917.050931,0.03212,0.0107,2.35996,1.85096,AutoRegressive,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25916,0.994962,2013-12-28 21:43:50,10.515294,2599.999304,702948.0,25.869017,73917.050931,0.03212,0.0107,2.35996,1.85096,AutoRegressive,3,2
25917,0.994962,2013-12-28 21:48:50,10.515294,2599.999304,702948.0,25.869017,73917.050931,0.03212,0.0107,2.35996,1.85096,AutoRegressive,3,2
25918,0.994962,2013-12-28 21:53:50,10.515294,2599.999304,702948.0,25.869017,73917.050931,0.03212,0.0107,2.35996,1.85096,AutoRegressive,3,2
25919,0.994962,2013-12-28 21:58:50,10.515294,2599.999304,702948.0,25.869017,73917.050931,0.03212,0.0107,2.35996,1.85096,AutoRegressive,3,2


In [54]:
finalar.to_csv(r'C:/Users/jahna/Downloads/data/Amazon EC2 Instance Comparision1.csv', index = False)